# 蒼狼 AI V2.2 可觀測性分析平台 - Colab 一鍵啟動

歡迎使用蒼狼 AI V2.2！本 Colab 筆記本旨在引導您在 Google Colaboratory 環境中快速設定並啟動應用程式的後端與前端服務。

**請依照下方儲存格的指引完成設定並啟動應用程式。**

In [ ]:
#@title 0. 指定要部署的程式碼分支
#@markdown --- 
#@markdown 請輸入要從 GitHub 下載的程式碼分支名稱。通常保持預設值 (`main`) 即可。
#@markdown - **main**: 代表最穩定的官方發布版。
#@markdown - **feat/...**: 代表正在開發中的新功能版。
#@markdown - **dev**: 代表開發整合分支。
branch_name = "main" #@param {type:"string"}
print(f"將從 GitHub 的 '{branch_name}' 分支下載最新的程式碼。")

In [ ]:
#@title 1. 選擇操作模式
#@markdown --- 
#@markdown **重要提示：** 在執行此儲存格之前，請務必根據您選擇的模式，在 Colab 的「密鑰 (Secrets)」管理器中設定必要的密鑰。詳細說明請參閱下方的「Colab Secrets 環境設定指南」。
#@markdown 
#@markdown 請選擇操作模式：
#@markdown - **暫存模式 (資料不會保存)**：無需 Google Drive 授權，服務將使用臨時儲存空間，所有資料在 Colab 會話結束後將會遺失。適合快速體驗或測試。
#@markdown - **持久模式 (需 Google Drive 授權)**：需要授權掛載您的 Google Drive，應用程式資料 (如資料庫檔案) 將儲存在您的 Drive 中，實現資料持久化。建議用於實際使用。
operation_mode = "transient" #@param ["暫存模式 (資料不會保存)", "持久模式 (需 Google Drive 授權)"]

import os
# 根據中文選項設定 OPERATION_MODE 環境變數
if operation_mode == "持久模式 (需 Google Drive 授權)":
    os.environ['OPERATION_MODE'] = "persistent"
elif operation_mode == "暫存模式 (資料不會保存)":
    os.environ['OPERATION_MODE'] = "transient"
else:
    # 預設或意外情況，也設為 transient，但打印警告
    os.environ['OPERATION_MODE'] = "transient"
    print("警告：操作模式選擇出現非預期情況，已自動設為「暫存模式」。")

print(f"您選擇的操作模式是：{operation_mode}")
print(f"環境變數 OPERATION_MODE 已設定為: {os.getenv('OPERATION_MODE')}")

# --- 初始化與環境設定 --- 
print("INFO: 開始執行 Colab 啟動程序...")
import shutil
import time
import requests
from IPython.display import display, HTML

# --- 步驟 1.1: 掛載 Google Drive (持久模式下) --- 
print("\nSTEP 1.1: 檢查是否需要掛載 Google Drive...")
if os.getenv('OPERATION_MODE') == "persistent":
    print("INFO: 持久模式 - 開始掛載 Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive') 
        print("SUCCESS: Google Drive 已成功掛載到 /content/drive")
    except Exception as e:
        print(f"ERROR: 掛載 Google Drive 失敗: {e}")
        print("請檢查彈出視窗是否已正確授權，或手動執行左側「掛載 Drive」按鈕。")
        print("持久模式下，Drive 掛載失敗將導致後續資料庫路徑等設定錯誤，建議解決後重試。")
        raise SystemExit("Google Drive 掛載失敗，請處理後重試。") 
else:
    print("INFO: 暫存模式 - 跳過 Google Drive 掛載。")

## Colab Secrets 環境設定指南

在執行 Colab 筆記本的初始化儲存格（特別是選擇「持久模式」時）之前，您需要在 Colab 的「密鑰 (Secrets)」管理器中設定以下環境變數。這樣可以安全地儲存敏感資訊，例如 API 金鑰和資料夾 ID，而不會將它們直接寫入程式碼中。

**如何設定 Colab Secrets：**
1. 開啟 Colab 筆記本後，點擊左側工具欄中的「鑰匙」圖示 (密鑰)。
2. 點擊「新增密鑰」。
3. 輸入密鑰的「名稱」(例如 `COLAB_GOOGLE_API_KEY`) 和對應的「值」。
4. 確保「筆記本訪問權限」已開啟 (滑塊按鈕應為藍色)。
5. 為下表列出的每個必要或建議的密鑰重複此過程。

**密鑰詳細說明表：**

| 密鑰名稱                             | 建議值/格式                                       | 持久模式必需 | 暫存模式必需 | 說明                                                                                                                               |
| :----------------------------------- | :------------------------------------------------ | :----------- | :----------- | :--------------------------------------------------------------------------------------------------------------------------------- |
| `COLAB_GOOGLE_API_KEY`               | 您的 Google AI (例如 Gemini) API 金鑰               | 是           | 建議         | 用於蒼狼 AI 的核心分析功能。即使在暫存模式下，若要使用 AI 分析，此金鑰也是必要的。                                                                     |
| `GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT`| Google Cloud 服務帳號金鑰的 JSON **內容**         | 是           | 否           | 用於授權 Colab 存取您的 Google Drive (讀取報告、寫入資料庫等)。請貼上 JSON 檔案的**完整內容**，而非檔案路徑。此服務帳號需有 Drive 的讀寫權限。        |
| `WOLF_IN_FOLDER_ID`                  | Google Drive 中「待處理報告」資料夾的 ID            | 是           | 否           | 指定 Google Drive 中存放上傳等待分析的 PDF 報告的資料夾 ID。例如：`1aBcDeFgHiJkLmNoPqRsTuVwXyZ_12345`                                          |
| `WOLF_PROCESSED_FOLDER_ID`           | Google Drive 中「已處理報告」資料夾的 ID            | 是           | 否           | 指定 Google Drive 中存放已完成分析並已存檔的 PDF 報告的資料夾 ID。例如：`1bCdEfGhIjKlMnOpQrStUvWxYz_67890`                                          |
| `REPORTS_DB_FILENAME`                | 資料庫檔案名稱 (可選，預設 `reports.sqlite`)      | 否           | 否           | （可選）持久模式下，報告資料庫檔案儲存於 Drive 的 `wolf_AI_data` 資料夾中；暫存模式下則在 Colab 本地。可自訂檔名。                                     |
| `PROMPTS_DB_FILENAME`                | 프롬프트 資料庫檔案名稱 (可選，預設 `prompts.sqlite`) | 否           | 否           | （可選）持久模式下，프롬프트 資料庫檔案儲存於 Drive 的 `wolf_AI_data` 資料夾中；暫存模式下則在 Colab 本地。可自訂檔名。                                |

**重要注意事項：**
*   對於「持久模式」，`GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT`、`WOLF_IN_FOLDER_ID` 和 `WOLF_PROCESSED_FOLDER_ID` 是**絕對必要**的。缺少任何一個都可能導致程式執行失敗。
*   `COLAB_GOOGLE_API_KEY` 對於應用程式的核心 AI 功能至關重要，強烈建議在兩種模式下都進行設定。
*   資料夾 ID 可以從 Google Drive 資料夾的 URL 中獲取 (URL 末尾的那串字符)。
*   建議在您的 Google Drive 根目錄下建立一個名為 `wolf_AI_data` 的主資料夾，並在其內部建立 `wolf_in` (用於存放待處理報告) 和 `processed` (用於存放已處理報告) 子資料夾。然後將這些子資料夾的 ID 用於 `WOLF_IN_FOLDER_ID` 和 `WOLF_PROCESSED_FOLDER_ID`。
*   設定完所有必要的密鑰後，再執行後續的程式碼儲存格。

In [ ]:
#@title 2. 檢查 Colab Secrets 設定情況
#@markdown --- 
#@markdown 此儲存格會檢查您在 Colab「密鑰 (Secrets)」管理器中設定的環境變數是否符合所選操作模式的要求。
#@markdown **在執行此儲存格前，請確保已設定必要的密鑰。**

import os
print("INFO: 開始檢查 Colab Secrets 設定情況...")
all_secrets_ok = True
operation_mode_env = os.getenv('OPERATION_MODE') # 從上一個儲存格設定的環境變數讀取

print(f"當前操作模式為: {operation_mode_env}")

# 檢查 COLAB_GOOGLE_API_KEY (AI 服務金鑰)
print("\n檢查 COLAB_GOOGLE_API_KEY (AI 服務金鑰)...")
google_api_key_env = os.getenv('COLAB_GOOGLE_API_KEY')
if not google_api_key_env:
    print("  ⚠️ COLAB_GOOGLE_API_KEY 未設定。AI 分析功能可能無法使用。建議在 Colab Secrets 中設定此金鑰。")
    if operation_mode_env == "persistent":
        print("  ❌ 持久模式下，COLAB_GOOGLE_API_KEY 未設定將導致AI功能無法運作!")
        all_secrets_ok = False # 在持久模式下，若無API金鑰則視為不完整
else:
    print("  ✅ COLAB_GOOGLE_API_KEY 已設定。")

# 持久模式下的特定檢查
if operation_mode_env == "persistent":
    print("\nINFO: 持久模式 - 檢查 Google Drive 相關 Colab Secrets...")
    secrets_to_check_persistent = [
        'GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT',
        'WOLF_IN_FOLDER_ID',
        'WOLF_PROCESSED_FOLDER_ID'
    ]
    missing_secrets_persistent = []
    for secret_name in secrets_to_check_persistent:
        secret_value = os.getenv(secret_name)
        if not secret_value:
            missing_secrets_persistent.append(secret_name)
            print(f"  ❌ {secret_name} 未設定或為空值！")
        else:
            print(f"  ✅ {secret_name} 已設定。")

    if missing_secrets_persistent:
        print("\n⚠️ **警告：持久模式下，一個或多個必要的 Google Drive Colab Secret 未被正確設定！**")
        print("缺失的 Secrets 列表:")
        for ms in missing_secrets_persistent:
            print(f"    - {ms}")
        all_secrets_ok = False
    else:
        print("\n✅ 持久模式下所有必要的 Google Drive Colab Secrets 看起來都已設定。")

    # 設定持久模式下的資料庫路徑 (Google Drive)
    if all_secrets_ok: # 只有在所有 Drive 相關 Secrets 都OK的情況下才設定路徑
        drive_mounted_base_path = "/content/drive/MyDrive/wolf_AI_data" 
        reports_db_file = os.getenv('REPORTS_DB_FILENAME', 'reports.sqlite')
        prompts_db_file = os.getenv('PROMPTS_DB_FILENAME', 'prompts.sqlite')
        os.environ['REPORTS_DB_PATH'] = os.path.join(drive_mounted_base_path, reports_db_file)
        os.environ['PROMPTS_DB_PATH'] = os.path.join(drive_mounted_base_path, prompts_db_file)
        print(f"環境變數 REPORTS_DB_PATH 已設定為 (持久模式): {os.getenv('REPORTS_DB_PATH')}")
        print(f"環境變數 PROMPTS_DB_PATH 已設定為 (持久模式): {os.getenv('PROMPTS_DB_PATH')}")
    else:
        print("ERROR: 持久模式下由於 Secrets 未完整設定，無法配置 Google Drive 中的資料庫路徑。")

elif operation_mode_env == "transient":
    print("\nINFO: 暫存模式 - 跳過 Google Drive 相關 Colab Secrets 的深入檢查。")
    # 設定暫存模式下的資料庫路徑 (Colab 本地)
    project_parent_dir_for_db = "/content/wolf_project"
    project_name_for_db = "wolfAI_v1" # 應與後續 git clone 的專案名稱一致
    base_data_path_for_db = os.path.join(project_parent_dir_for_db, project_name_for_db, 'data')
    reports_db_file_transient = os.getenv('REPORTS_DB_FILENAME', 'session_reports.sqlite')
    prompts_db_file_transient = os.getenv('PROMPTS_DB_FILENAME', 'session_prompts.sqlite')
    os.environ['REPORTS_DB_PATH'] = os.path.join(base_data_path_for_db, reports_db_file_transient)
    os.environ['PROMPTS_DB_PATH'] = os.path.join(base_data_path_for_db, prompts_db_file_transient)
    print(f"環境變數 REPORTS_DB_PATH 已設定為 (暫存模式): {os.getenv('REPORTS_DB_PATH')}")
    print(f"環境變數 PROMPTS_DB_PATH 已設定為 (暫存模式): {os.getenv('PROMPTS_DB_PATH')}")
else:
    print("\nERROR: 未知的操作模式，無法正確檢查 Secrets 或設定資料庫路徑。")
    all_secrets_ok = False

if not all_secrets_ok:
    print("\n❌ **錯誤：Colab Secrets 設定不完整或不正確（根據所選操作模式），無法繼續。**")
    print("請返回 Colab 的「密鑰 (Secrets)」管理器進行設定，或調整操作模式，然後重新執行此初始化流程。")
    raise SystemExit("Colab Secrets 設定不完整，請檢查後重試。")
else:
    print("\n✅ Colab Secrets 設定檢查完畢。可以繼續執行專案部署。")

# 移除舊的 print 提示，因為已經整合到新的 Markdown 儲存格中
# print("\nSTEP 2: 設定 Google API 金鑰與服務帳號憑證 (Colab Secrets)") ... (相關 print 語句已刪除)
# print("\nSTEP 3: 設定蒼狼 AI 資料夾 ID (Colab Secrets)") ... (相關 print 語句已刪除)

In [ ]:
#@title 3. 選擇啟動模式
#@markdown --- 
#@markdown 請選擇應用程式的啟動模式：
#@markdown - **一般模式 (日誌寫入檔案)**：應用程式日誌將寫入到專案資料夾內的日誌檔案 (`backend.log`, `frontend.log`)。適合常規運行。
#@markdown - **除錯模式 (日誌即時輸出)**：應用程式日誌將直接輸出到此 Colab 儲存格的輸出區域，方便即時追蹤和除錯。
MODE_CHOICE = "一般模式 (日誌寫入檔案)" #@param ["一般模式 (日誌寫入檔案)", "除錯模式 (日誌即時輸出)"]

script_mode_param = "normal" # 預設值
if MODE_CHOICE == "除錯模式 (日誌即時輸出)":
    script_mode_param = "debug"
elif MODE_CHOICE == "一般模式 (日誌寫入檔案)":
    script_mode_param = "normal"

print(f"您選擇的啟動模式是：{MODE_CHOICE}")
print(f"將傳遞給啟動腳本的模式參數是：--mode={script_mode_param}")

# 將 script_mode_param 存儲起來，或者直接在後續命令中使用
# 為了清晰，我們可以在環境中設定一個變數，儘管 start.sh 更傾向於接收命令行參數
os.environ['APP_LAUNCH_MODE'] = script_mode_param 

In [ ]:
#@title 4. 複製專案程式碼並啟動服務
#@markdown --- 
#@markdown 此儲存格將執行以下操作：
#@markdown 1. 從 GitHub 複製蒼狼 AI V2.2 的最新程式碼 (根據步驟 0 選擇的分支)。
#@markdown 2. 若為「暫存模式」，則在複製下來的專案內建立本地資料庫所需的 `data` 資料夾。
#@markdown 3. 執行主控啟動腳本 `scripts/start.sh`，該腳本會負責安裝依賴、啟動後端 FastAPI 和前端 Next.js 服務。
#@markdown 4. 進行健康檢查，確認服務是否成功啟動。
#@markdown 
#@markdown **執行時間提醒：** 此過程可能需要幾分鐘時間，特別是首次執行時，因為需要下載程式碼和安裝依賴。請耐心等候。

import os
import shutil
import time
import requests
from IPython.display import display, HTML

print("\nSTEP 4.1: 複製專案程式碼...")
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git" # 確保這是正確的 V2.2 倉庫 URL
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1" # 建議與倉庫名稱一致
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)

if os.path.exists(FULL_PROJECT_PATH):
    print(f"偵測到已存在的專案資料夾 '{FULL_PROJECT_PATH}'，將先移除以進行全新複製...")
    shutil.rmtree(FULL_PROJECT_PATH)
    print(f"舊專案資料夾 '{FULL_PROJECT_PATH}' 已成功移除。")

os.makedirs(PROJECT_PARENT_DIR, exist_ok=True)
print(f"開始從 {GIT_REPO_URL} 的 '{branch_name}' 分支複製專案到 {FULL_PROJECT_PATH}...") # branch_name 來自步驟 0
clone_command = f"git clone --depth 1 -b {branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}"
clone_status = os.system(clone_command)

if clone_status != 0 or not os.path.isdir(FULL_PROJECT_PATH):
    print(f"ERROR: 專案複製失敗。返回碼: {clone_status}。請檢查 GIT_REPO_URL ('{GIT_REPO_URL}') 和分支名稱 ('{branch_name}') 是否正確以及您的網路連線。")
    raise SystemExit("專案複製失敗。")
else:
    print(f"SUCCESS: 專案已成功從 '{branch_name}' 分支複製到 '{FULL_PROJECT_PATH}' 目錄。")
    
    operation_mode_env = os.getenv('OPERATION_MODE')
    if operation_mode_env == 'transient':
        path_for_transient_db_data = os.path.join(FULL_PROJECT_PATH, 'data')
        if not os.path.exists(path_for_transient_db_data):
            os.makedirs(path_for_transient_db_data)
            print(f"為暫存模式在專案內創建了資料目錄: {path_for_transient_db_data}")

    print("\nSTEP 4.2: 執行主控啟動腳本 scripts/start.sh...")
    start_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'start.sh')
    app_launch_mode = os.getenv('APP_LAUNCH_MODE', 'normal') # 從環境變數讀取，若未設定則預設 normal

    if os.path.exists(start_script_path):
        print(f"賦予腳本執行權限: chmod +x {start_script_path}")
        os.system(f"chmod +x {start_script_path}")
        print(f"執行腳本: {start_script_path} --mode={app_launch_mode}")
        start_script_command = f"{start_script_path} --mode={app_launch_mode}"
        exit_code = os.system(start_script_command) # 執行腳本並等待其完成
        if exit_code != 0:
            print(f"ERROR: 啟動腳本 {start_script_path} 執行失敗，返回碼: {exit_code}。請檢查上方日誌。")
            # 不在此處終止，允許進行後續的健康檢查以獲取更多資訊
        else:
            print(f"INFO: 啟動腳本 {start_script_path} 已執行。")
    else:
        print(f"ERROR: 主控啟動腳本 {start_script_path} 未找到！無法繼續。")
        raise SystemExit(f"啟動腳本 {start_script_path} 未找到。")

    print("\nSTEP 4.3: 健康度偵測 (Health Check) - 請耐心等待約 1-2 分鐘...")
    BACKEND_SERVICE_PORT = 8000
    FRONTEND_SERVICE_PORT = 3000
    max_wait_duration_seconds = 180 # 增加等待時間以應對較慢的啟動
    check_interval_seconds = 15
    start_check_time = time.time()
    backend_service_ready, frontend_service_ready = False, False

    while time.time() - start_check_time < max_wait_duration_seconds:
        if not backend_service_ready:
            try:
                response_backend = requests.get(f"http://localhost:{BACKEND_SERVICE_PORT}/api/health", timeout=10)
                if response_backend.status_code == 200 and response_backend.json().get('status') == 'OK':
                    backend_service_ready = True
                    print(f"[{time.strftime('%H:%M:%S')}] ✅ 後端服務 READY. 狀態: {response_backend.json()}")
                else:
                    print(f"[{time.strftime('%H:%M:%S')}] ⏳ 後端服務未就緒 (HTTP {response_backend.status_code}) - {response_backend.text[:100]}")
            except requests.RequestException:
                print(f"[{time.strftime('%H:%M:%S')}] ⏳ 後端服務未就緒 (連接失敗)")
        
        if not frontend_service_ready:
            try:
                response_frontend = requests.get(f"http://localhost:{FRONTEND_SERVICE_PORT}", timeout=10)
                if response_frontend.status_code == 200:
                    frontend_service_ready = True
                    print(f"[{time.strftime('%H:%M:%S')}] ✅ 前端服務 READY.")
                else:
                    print(f"[{time.strftime('%H:%M:%S')}] ⏳ 前端服務未就緒 (HTTP {response_frontend.status_code})")
            except requests.RequestException:
                print(f"[{time.strftime('%H:%M:%S')}] ⏳ 前端服務未就緒 (連接失敗)")

        if backend_service_ready and frontend_service_ready: 
            print("\n🎉 太棒了！後端和前端服務均已成功啟動並通過健康檢查！")
            break
        
        if time.time() - start_check_time >= max_wait_duration_seconds: 
            print("\n⏱️ 服務啟動或健康檢查超時。")
            if not backend_service_ready: print("   - 後端服務未能達到健康狀態。")
            if not frontend_service_ready: print("   - 前端服務未能達到健康狀態。")
            print("   請檢查上方 start.sh 腳本的輸出，以及可能的 backend.log 和 frontend.log (在一般模式下)。")
            break
            
        print(f"   ...等待 {check_interval_seconds} 秒後重試...")
        time.sleep(check_interval_seconds)

    if backend_service_ready and frontend_service_ready:
        border = "="*70
        print(f"\n{border}")
        display(HTML("<h2>✅ 應用程式已就緒，可以開始使用了！</h2>"))
        print("🔗 前端應用程式訪問網址 (Colab Proxy URL):")
        print(f"   請在本儲存格的輸出區域上方尋找由 Colab 自動生成的、形如 https://*.googleusercontent.com/ 的公開連結。")
        print(f"   這個連結指向您的前端應用程式 (端口 {FRONTEND_SERVICE_PORT})。")
        display(HTML(f"<p style='font-size:1.1em;'>點擊 Colab 在此儲存格輸出上方提供的 <code>https://....googleusercontent.com/</code> 公開連結即可訪問。</p>"))
        print(f"{border}\n")
    else:
        print("\n❌ 很抱歉，一個或多個服務未能成功啟動或通過健康檢查。")
        print("   請仔細檢查上方的日誌輸出以了解詳細錯誤信息。")
        if app_launch_mode == 'normal':
             backend_log_path = os.path.join(FULL_PROJECT_PATH, 'backend.log')
             frontend_log_path = os.path.join(FULL_PROJECT_PATH, 'frontend.log')
             print(f"   後端服務日誌檔案可能位於: {backend_log_path}")
             print(f"   前端服務日誌檔案可能位於: {frontend_log_path}")
             print("   您可以點擊左側「檔案」圖示，導航到相應路徑查看日誌內容。")
        else: # debug mode
             print("   由於處於除錯模式，服務日誌應已直接混合輸出到本儲存格。")

print("\nINFO: Colab 啟動程序執行完畢。")